In [1]:
import numpy as np
import pandas as pd
import main
fake_data_wc = pd.read_csv('fake_data/fake_wc_res.csv', delimiter=';')
fake_data_wc.head()

,ID,X,Y,WC
0,0,348202,6460000,1.700000e+38
1,1,348202,6460000,1.700000e+38
2,2,348202,6460000,1.700000e+38
3,3,348202,6460000,1.700000e+38
4,4,348202,6460000,1.700000e+38


In [2]:
# Конвертируем форматы
fake_data_wc['WC'] = fake_data_wc['WC'].astype('float64')
fake_data_wc['ID'] = fake_data_wc['ID'].astype('str')
fake_data_wc['WELL_NAME'] = 'fake_' + fake_data_wc['ID']
del fake_data_wc['ID']
fake_data_wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5399 entries, 0 to 5398
Data columns (total 4 columns):
X            5399 non-null int64
Y            5399 non-null int64
WC           5399 non-null float64
WELL_NAME    5399 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 168.8+ KB


In [3]:
# Удаляем значения зашедшие за край
fake_data_wc = fake_data_wc.drop(fake_data_wc[fake_data_wc['WC'] > 1].index)
fake_data_wc.head()

,X,Y,WC,WELL_NAME
24,348302,6460000,0.39,fake_24
25,348302,6460000,0.39,fake_25
26,348302,6460000,0.39,fake_26
27,348302,6460000,0.39,fake_27
28,348302,6460000,0.39,fake_28


In [4]:
fake_las_out = pd.read_csv('fake_data/fake_las_out.csv', delimiter=';')
fake_las_out.head()

,WELL_NAME,DEPTH,APS
0,fake_0,2584.999314,0.380130
1,fake_0,2585.121187,0.380130
2,fake_0,2585.121309,0.364062
3,fake_0,2585.243182,0.364062
4,fake_0,2585.243304,0.309785


In [5]:
fake_bottom = pd.read_csv('fake_data/fake_bottom_res.csv', delimiter=';')
fake_bottom['ID'] = fake_bottom['ID'].astype('str')
fake_bottom['WELL_NAME'] = 'fake_' + fake_bottom['ID']
fake_bottom = fake_bottom.rename(columns={'DEPTH': 'DEPTH_BOTTOM'})
del fake_bottom['ID']
del fake_bottom['X']
del fake_bottom['Y']
fake_bottom.head()

,DEPTH_BOTTOM,WELL_NAME
0,2601.50,fake_0
1,2600.09,fake_1
2,2598.85,fake_2
3,2598.79,fake_3
4,2599.31,fake_4


In [6]:
fake_top = pd.read_csv('fake_data/fake_top_res.csv', delimiter=';')
fake_top['ID'] = fake_top['ID'].astype('str')
fake_top['WELL_NAME'] = 'fake_' + fake_top['ID']
fake_top = fake_top.rename(columns={'DEPTH': 'DEPTH_TOP'})
del fake_top['ID']
del fake_top['X']
del fake_top['Y']
fake_top.head()

,DEPTH_TOP,WELL_NAME
0,2585.02,fake_0
1,2582.98,fake_1
2,2581.22,fake_2
3,2580.52,fake_3
4,2580.87,fake_4


In [7]:
fake_data = pd.merge(fake_data_wc, fake_las_out, on='WELL_NAME')
fake_data = pd.merge(fake_data, fake_top, on='WELL_NAME') 
fake_data = pd.merge(fake_data, fake_bottom, on='WELL_NAME')
fake_data.head()

,X,Y,WC,WELL_NAME,DEPTH,APS,DEPTH_TOP,DEPTH_BOTTOM
0,348302,6460000,0.39,fake_24,2588.656453,0.274434,2588.27,2603.89
1,348302,6460000,0.39,fake_24,2588.771455,0.274434,2588.27,2603.89
2,348302,6460000,0.39,fake_24,2588.771570,0.274434,2588.27,2603.89
3,348302,6460000,0.39,fake_24,2588.886333,0.191055,2588.27,2603.89
4,348302,6460000,0.39,fake_24,2588.886448,0.101674,2588.27,2603.89


In [16]:
from scipy import interpolate
well_name_list = fake_data['WELL_NAME'].value_counts().index.tolist()
x_values = []
y_values = []
well_values = []
count_val = 100
well_count = 0
min_count_val_in_data = 10
for well_name in well_name_list:
    if well_count%500 == 0:
        print(well_count, ' of ', len(well_name_list))
    data_well =  fake_data[fake_data['WELL_NAME'] == well_name]
    bottom = data_well['DEPTH_BOTTOM'].tolist()[0]
    top = data_well['DEPTH_TOP'].tolist()[0]
    # print('top = ', top)
    # print('bottom =', bottom)    
    data_well_by_bound = data_well[(data_well['DEPTH'] >= top) & (data_well['DEPTH'] <= bottom)]
    x_arr = data_well_by_bound['DEPTH']
    y_arr = data_well_by_bound['APS']
    
    if (len(x_arr) < min_count_val_in_data):
            continue
    
    # print ('length of array depth', len(x_arr))
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPTH'].min()
    h_end = data_well_by_bound['DEPTH'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['WC'].tolist()[0])
    well_values.append(data_well['WELL_NAME'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  5322
500  of  5322
1000  of  5322
1500  of  5322
2000  of  5322
2500  of  5322
3000  of  5322
3500  of  5322
4000  of  5322
4500  of  5322
5000  of  5322
end fill


In [17]:
x_train = pd.DataFrame(x_values, columns=['aps' + str(z) for z in range(0, len(x_values[0]))])
y_train = pd.DataFrame(y_values, columns=['wc'])
well_train = pd.DataFrame(well_values, columns=['well_name'])

save_train = pd.merge(well_train, y_train, left_index = True, right_index = True)
save_train = pd.merge(save_train, x_train, left_index = True, right_index = True)

save_train.to_csv('fake_data/fake_data_collect_aps.csv', sep = ';', index = False)

In [18]:
save_train.head()

,well_name,wc,aps0,aps1,aps2,aps3,aps4,aps5,aps6,aps7,...,aps90,aps91,aps92,aps93,aps94,aps95,aps96,aps97,aps98,aps99
0,fake_1538,0.33,0.561129,0.563750,0.589631,0.592500,0.590609,0.590920,0.622807,0.599743,...,0.369530,0.370963,0.353319,0.338751,0.358087,0.298228,0.336996,0.326959,0.359809,0.303056
1,fake_1823,0.36,0.771901,0.844685,0.825432,0.766603,0.900882,1.035077,0.867133,0.819914,...,0.685977,0.574345,0.504850,0.492639,0.389353,0.543809,0.511795,0.554623,0.426828,0.423948
2,fake_1560,0.23,0.727815,0.662211,0.728361,0.820049,0.783995,0.774724,0.700156,0.771722,...,0.634780,0.610779,0.616523,0.632861,0.644463,0.725761,0.739720,0.761486,0.786858,0.815196
3,fake_814,0.26,0.538646,0.627068,0.847221,0.804922,0.849598,0.876741,0.866902,0.890146,...,0.172137,0.197857,0.192303,0.143776,0.164691,0.164772,0.173827,0.203548,0.120589,0.113305
4,fake_1403,0.46,0.532483,0.479290,0.334221,0.254886,0.275549,0.870153,0.878936,0.881105,...,0.074392,0.040271,0.062428,0.057433,0.054835,0.028831,0.096805,0.076488,0.011525,0.003258


In [19]:
import numpy as np
import pandas as pd
import main
fake_data_wc = pd.read_csv('fake_data/fake_wc_res.csv', delimiter=';')
fake_data_wc.head()

# Конвертируем форматы
fake_data_wc['WC'] = fake_data_wc['WC'].astype('float64')
fake_data_wc['ID'] = fake_data_wc['ID'].astype('str')
fake_data_wc['WELL_NAME'] = 'fake_' + fake_data_wc['ID']
del fake_data_wc['ID']
fake_data_wc.info()


# Удаляем значения зашедшие за край
fake_data_wc = fake_data_wc.drop(fake_data_wc[fake_data_wc['WC'] > 1].index)
fake_data_wc.head()

fake_las_out = pd.read_csv('fake_data/fake_las_kgl_out.csv', delimiter=';')
fake_las_out = fake_las_out.rename(columns={'DEPT': 'DEPTH'})
fake_las_out = fake_las_out.rename(columns={'well_name': 'WELL_NAME'})
fake_las_out = fake_las_out.rename(columns={'kgl': 'KGL'})
fake_las_out.head()

fake_bottom = pd.read_csv('fake_data/fake_bottom_res.csv', delimiter=';')
fake_bottom['ID'] = fake_bottom['ID'].astype('str')
fake_bottom['WELL_NAME'] = 'fake_' + fake_bottom['ID']
fake_bottom = fake_bottom.rename(columns={'DEPTH': 'DEPTH_BOTTOM'})
del fake_bottom['ID']
del fake_bottom['X']
del fake_bottom['Y']
fake_bottom.head()

fake_top = pd.read_csv('fake_data/fake_top_res.csv', delimiter=';')
fake_top['ID'] = fake_top['ID'].astype('str')
fake_top['WELL_NAME'] = 'fake_' + fake_top['ID']
fake_top = fake_top.rename(columns={'DEPTH': 'DEPTH_TOP'})
del fake_top['ID']
del fake_top['X']
del fake_top['Y']
fake_top.head()

fake_data = pd.merge(fake_data_wc, fake_las_out, on='WELL_NAME')
fake_data = pd.merge(fake_data, fake_top, on='WELL_NAME') 
fake_data = pd.merge(fake_data, fake_bottom, on='WELL_NAME')
fake_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5399 entries, 0 to 5398
Data columns (total 4 columns):
X            5399 non-null int64
Y            5399 non-null int64
WC           5399 non-null float64
WELL_NAME    5399 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 168.8+ KB


,X,Y,WC,WELL_NAME,DEPTH,KGL,DEPTH_TOP,DEPTH_BOTTOM
0,348302,6460000,0.39,fake_24,2588.656453,0.343177,2588.27,2603.89
1,348302,6460000,0.39,fake_24,2588.811598,0.343177,2588.27,2603.89
2,348302,6460000,0.39,fake_24,2588.811753,0.337165,2588.27,2603.89
3,348302,6460000,0.39,fake_24,2588.966872,0.337165,2588.27,2603.89
4,348302,6460000,0.39,fake_24,2588.967028,0.329024,2588.27,2603.89


In [20]:
from scipy import interpolate
well_name_list = fake_data['WELL_NAME'].value_counts().index.tolist()
x_values = []
y_values = []
well_values = []
count_val = 100
well_count = 0
for well_name in well_name_list:
    if well_count%500 == 0:
        print(well_count, ' of ', len(well_name_list))
    data_well =  fake_data[fake_data['WELL_NAME'] == well_name]
    bottom = data_well['DEPTH_BOTTOM'].tolist()[0]
    top = data_well['DEPTH_TOP'].tolist()[0]
    # print('top = ', top)
    # print('bottom =', bottom)    
    data_well_by_bound = data_well[(data_well['DEPTH'] >= top) & (data_well['DEPTH'] <= bottom)]
    x_arr = data_well_by_bound['DEPTH']
    #y_arr = data_well_by_bound['APS']
    y_arr = data_well_by_bound['KGL']
    
    
    # print ('length of array depth', len(x_arr))
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPTH'].min()
    h_end = data_well_by_bound['DEPTH'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['WC'].tolist()[0])
    well_values.append(data_well['WELL_NAME'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  5322
500  of  5322
1000  of  5322
1500  of  5322
2000  of  5322
2500  of  5322
3000  of  5322
3500  of  5322
4000  of  5322
4500  of  5322
5000  of  5322
end fill


In [21]:
x_train = pd.DataFrame(x_values, columns=['kgl' + str(z) for z in range(0, len(x_values[0]))])
y_train = pd.DataFrame(y_values, columns=['wc'])
well_train = pd.DataFrame(well_values, columns=['well_name'])

save_train = pd.merge(well_train, y_train, left_index = True, right_index = True)
save_train = pd.merge(save_train, x_train, left_index = True, right_index = True)

save_train.to_csv('fake_data/fake_data_collect_kgl.csv', sep = ';', index = False)

In [23]:
save_train.shape

(5322, 102)